## **앱스토어 리뷰 감성 분석**

### 문제정의
각 앱들의 리뷰들과 평점이 양의 관계에 있다고 가정하고 학습을 진행하여 평점을 예측

### 데이터 불러오기

tsv 파일 불러오기

In [2]:
def get_data_split(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        docs = [doc.lower().replace('\n','').split('\t') for doc in f.readlines()]
        docs = [doc for doc in docs if len(doc) == 3]
        
    if not docs:
        app_ids, texts, ratings = [], [], []

    app_ids, texts, ratings = zip(*docs)
    return list(app_ids), list(texts), list(ratings)

In [3]:
train_app_ids, train_texts, train_ratings = get_data_split('./data/train_docs.txt')
test_app_ids, test_texts, test_ratings = get_data_split('./data/test_docs.txt')

json 파일 불러오기

In [1]:
import ujson

def get_data_json(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        texts = file.read()
        data = ujson.loads(texts)
        app_ids_list = list(data.keys())
    return data, app_ids_list

In [55]:
train_data, train_app_id_list = get_data_json('./data/train_json.txt')
test_data, test_app_id_list = get_data_json('./data/test_json.txt')

### 데이터 전처리: 형태소 분석
1. 축약어, 오타:
골치거리임, 전형적인 한국 사전에 없는 단어들도 많음, 사전에 없는 단어를 트위터사전에 넣어야 할 듯
2. 띄어쓰기가 불분명함

$\Rightarrow$ data Cleaning class 필요성 있음

### 데이터 후처리

데이터 후처리란 형태소 분석기를 거친 후에 이상한 단어들, 신조어 등을 처리하는 작업


각 앱별로 공백문자를 나눠서 공통된 단어를 추출해보기? 앱별로 공통으로 많이 이야기 하는 단어들이 제대로 형태소 분석이 되는지 테스트 하기 위해 단어들을 추출해본다

문제점: 일일이 확인 하기 어려움

In [2]:
from collections import Counter, defaultdict
from konlpy.tag import Twitter, Komoran, Mecab
from utils import Unknown_words, Post_ma
twitter = Twitter()
komoran = Komoran()
mecab = Mecab()

## 형태소 분석 후처리

### Mecab

In [4]:
train_ma_docs, app_id_list = get_data_json('./data/ma_train_json_mecab.txt')

### Unknown에 대한 처리(Mecab) [잠정보류]
부호같은 거는 후처리를 안하고 그냥 제거하기로 

In [16]:
unknown_list = get_unknown_words(train_ma_docs, app_id_list)
len(unknown_list)

17237

In [17]:
unknown_list[40:60]

['ㅃㄴㅅ',
 '괜안네요',
 'ㅋㅋㅋㅋㅠ',
 'ㅋㅋㅋㅋㅋㅋ지혼자만',
 'ㅠㅠ위젯캘린더겸',
 'ㅋㅋ추천할만합니당',
 'ㅠㅠ당일',
 'ㅓㅓㅓㅓㅓ무',
 'ㅠㅠ큰맘먹고',
 '른어츨보다',
 '걍찍는사진도너무많아서',
 'ㅡㅡㅡㅡ수정좀요',
 '켯드라',
 'ㅇ었믄데',
 'ㅋㅋㅋ좋어요',
 '줜나게해가',
 '괜츈하더라구요',
 'ㅏ나가됨니다',
 'ㅋㅋㅋㅋㅋㅋㅋㅋ돌았나ㅋㅋㅋㅋㅋㅋㅋㅋ',
 'ㅠㅠ용랑때문에']

주로 말도 안되는 말들이나 ㅡ, ㅉ,ㅜ,ㅠ,ㅋ 앞이나 뒤에 붙어서 형태소 분석이 안된다.( 웃음(ㅋㅋ)이나 이모티콘(ㅜㅜ) 같은..) 이 단어들을 때주고 다시 넣어보자

처리가 되면 

{'기존 unknown':'형태소 분석 된 unknown'} 과 잔존 unkown_list 출력

앞에의 dictionary를 ma_unknown_rule.txt로 저장해서 두고주고 쓸것

In [19]:
unknown_rule, rest_unknown_list = remove_emo(unknown_list, mecab)

In [38]:
len(rest_unknown_list)

9981

---

## komoran
이게 제일 나은듯?


In [3]:
train_ma_docs, app_id_list = get_data_json('./data/ma_train_json_komoran.txt')

'NA'확인

In [4]:
UNK = Unknown_words(train_ma_docs, app_id_list)

In [5]:
unknown_list = UNK.get_unknown_words()

Unknown: 70342, total: 173157


In [7]:
unknown_list[20:40]

['나또채금ㅇ야',
 '대박일텐뎈',
 '부족해요ㅠㅠ삭제',
 '플스겜',
 '굳굿궀귣귯귰',
 'ㅂ좋아요',
 '왜바뀌고나서이따구여...전껄로돌려쥬....로지로지ㅜㅜㅜ',
 '강추합니다!!ㅎㅎㅎㅎㅎ',
 '번거로워요ㅜ',
 'ㅠㅠ자연의',
 '초기호ㅘ까지',
 '된다고하네요ㅜㅜ환불해주세요',
 '자동스크롤을',
 '우왕귿~!',
 '잘쓰고있었지말입니다ㅠㅠ',
 '중입니닿ㅎ아주',
 '너뮤',
 'ㅋㅋ성악가분이나',
 '재미이쩌영~~~',
 '구우웃....']

In [8]:
UNK.unknown_loc['ㅋㅋ성악가분이나']

[['371629295', (431, 23)]]

In [6]:
unknown_rule, rest_unknown_list = UNK.remove_no_meanings(unknown_list, komoran)
len(rest_unknown_list)

39287

이상하게 여기서 mecab쓰면 더 좋아짐

unknown_rule, rest_unknown_list = remove_emo(unknown_list, mecab)
len(rest_unknown_list)

리스트 형태로 바꿔줌

In [9]:
post_ma_list = UNK.shape_changer(unknown_rule)

('팅', 'IC'), ('김', 'NNP') >> (튕기, VV)

('팅', 'MAG'), ('기', 'VV')  >> (튕기, VV)

('배', 'VV'), ('ㄹ', 'ETM'), ('붕', 'NNG') >> ('밸런스', 'NNG'), ('붕괴', 'NNG')

등등..

형태소 분석기 돌리기 전에 우선 문법을 좀 고쳐야 되는ㄱ ㅓ아닌가 ..됬는데 --> 됐는데...

우선 단어 사전을 만들고 다시 unknown에 대해서 단어가 있으면 스페이싱을 하는 방법?

혹은 자음모음 나눠서 단어장에 유사도가 비슷하면 채택?

In [10]:
post_ma = Post_ma(tokenizer_name='komoran')

Load Complete!


In [10]:
# post_ma.update(post_ma_list, write=True)

In [10]:
# post_ma.load()

In [11]:
train_ma_docs = post_ma.replace_ma_docs(train_ma_docs, UNK.unknown_loc)

스위칭 됬는지 확인!

In [13]:
train_ma_docs['371629295'][431][23]

[['성악가', 'NNG'], ['분', 'XSN'], ['이나', 'JC']]

---

## Twitter

트위터가 생각보다 개판인듯... '내역'만 좀 어떻게 해보자

In [9]:
train_ma_docs, app_id_list = get_data_json('./data/ma_train_json_twitter.txt')

---

### 주요 품사별 추출 및 사전 만들기

MNG(일반 명사), NNP(고유 명사), VV(동사), VA(형용사)이며, 경우에 따라 MM(관형사), MAG(일반 부사), MAJ(접속 부사)

일단 노 추출하고 사전 만들어보자